In [ ]:
import os
from collections import defaultdict

# Đường dẫn đến các thư mục
base_dir = "./vietnamese"
train_dir = os.path.join(base_dir, "train_images")
test_dir = os.path.join(base_dir, "test_image")
unseen_test_dir = os.path.join(base_dir, "unseen_test_images")
label_dir = os.path.join(base_dir, "labels")

# Hàm lấy danh sách file ảnh (chỉ lấy file .jpg)
def get_image_files(directory):
    return sorted([f for f in os.listdir(directory) if f.endswith('.jpg')])

# Hàm lấy danh sách file nhãn (chỉ lấy file .txt)
def get_label_files(directory):
    return sorted([f for f in os.listdir(directory) if f.endswith('.txt')])

# Hàm trích xuất ID từ tên file (ví dụ: im0001.jpg -> 0001, gt_1.txt -> 1)
def extract_image_id(filename):
    return int(filename.replace('im', '').replace('.jpg', ''))

def extract_label_id(filename):
    return int(filename.replace('gt_', '').replace('.txt', ''))

# 1. Thống kê số lượng ảnh trong từng thư mục
train_images = get_image_files(train_dir)
test_images = get_image_files(test_dir)
unseen_test_images = get_image_files(unseen_test_dir)

print("=== Thống kê số lượng ảnh ===")
print(f"Số lượng ảnh trong train_images: {len(train_images)}")
print(f"Số lượng ảnh trong test_image: {len(test_images)}")
print(f"Số lượng ảnh trong unseen_test_images: {len(unseen_test_images)}")
print(f"Tổng số lượng ảnh: {len(train_images) + len(test_images) + len(unseen_test_images)}\n")

# 2. Thống kê số lượng nhãn
label_files = get_label_files(label_dir)
print("=== Thống kê nhãn ===")
print(f"Số lượng file nhãn: {len(label_files)}\n")

# 3. Kiểm tra khớp nối giữa ảnh và nhãn
# Tạo dictionary ánh xạ: image_id -> label_file
label_map = {extract_label_id(label): label for label in label_files}

# Tạo danh sách ID của ảnh
train_image_ids = [extract_image_id(img) for img in train_images]
test_image_ids = [extract_image_id(img) for img in test_images]
unseen_test_image_ids = [extract_image_id(img) for img in unseen_test_images]
all_image_ids = train_image_ids + test_image_ids + unseen_test_image_ids

# Tạo danh sách ID của nhãn
label_ids = [extract_label_id(label) for label in label_files]

# 4. Thống kê ảnh có nhãn và không có nhãn
images_with_label = []
images_without_label = []
for image_id in all_image_ids:
    if image_id in label_map:
        images_with_label.append(image_id)
    else:
        images_without_label.append(image_id)

labels_without_image = [label_id for label_id in label_ids if label_id not in all_image_ids]

print("=== Thống kê ảnh có nhãn và không có nhãn ===")
print(f"Số lượng ảnh có nhãn: {len(images_with_label)}")
print(f"Số lượng ảnh không có nhãn: {len(images_without_label)}")
print(f"Số lượng nhãn không có ảnh tương ứng: {len(labels_without_image)}\n")

# 5. Phân tích chi tiết từng thư mục
print("=== Phân tích chi tiết từng thư mục ===")
# Train images
train_with_label = [img_id for img_id in train_image_ids if img_id in label_map]
train_without_label = [img_id for img_id in train_image_ids if img_id not in label_map]
print("Thư mục train_images:")
print(f"  - Số lượng ảnh có nhãn: {len(train_with_label)}")
print(f"  - Số lượng ảnh không có nhãn: {len(train_without_label)}")
if train_without_label:
    print(f"  - Danh sách ảnh không có nhãn: {train_without_label}\n")
else:
    print("  - Không có ảnh nào thiếu nhãn.\n")

# Test images
test_with_label = [img_id for img_id in test_image_ids if img_id in label_map]
test_without_label = [img_id for img_id in test_image_ids if img_id not in label_map]
print("Thư mục test_image:")
print(f"  - Số lượng ảnh có nhãn: {len(test_with_label)}")
print(f"  - Số lượng ảnh không có nhãn: {len(test_without_label)}")
if test_without_label:
    print(f"  - Danh sách ảnh không có nhãn: {test_without_label}\n")
else:
    print("  - Không có ảnh nào thiếu nhãn.\n")

# Unseen test images
unseen_with_label = [img_id for img_id in unseen_test_image_ids if img_id in label_map]
unseen_without_label = [img_id for img_id in unseen_test_image_ids if img_id not in label_map]
print("Thư mục unseen_test_images:")
print(f"  - Số lượng ảnh có nhãn: {len(unseen_with_label)}")
print(f"  - Số lượng ảnh không có nhãn: {len(unseen_without_label)}")
if unseen_without_label:
    print(f"  - Danh sách ảnh không có nhãn: {unseen_without_label}\n")
else:
    print("  - Không có ảnh nào thiếu nhãn.\n")

# 6. Kiểm tra nhãn không có ảnh tương ứng
if labels_without_image:
    print("=== Nhãn không có ảnh tương ứng ===")
    print(f"Danh sách nhãn không có ảnh: {labels_without_image}\n")
else:
    print("=== Nhãn không có ảnh tương ứng ===")
    print("Không có nhãn nào thiếu ảnh.\n")

# 7. Kiểm tra lỗi trong file nhãn (nội dung rỗng, định dạng sai)
print("=== Kiểm tra lỗi trong file nhãn ===")
errors = []
for label_file in label_files:
    label_path = os.path.join(label_dir, label_file)
    with open(label_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        if not lines:
            errors.append(f"File nhãn {label_file} rỗng.")
            continue
        for line in lines:
            if '###' in line:
                continue
            parts = line.strip().split(',')
            if len(parts) < 9:  # 8 tọa độ + 1 văn bản
                errors.append(f"File nhãn {label_file} có dòng sai định dạng: {line.strip()}")
            else:
                # Kiểm tra tọa độ có phải là số không
                coords = parts[:8]
                try:
                    [int(coord) for coord in coords]
                except ValueError:
                    errors.append(f"File nhãn {label_file} có tọa độ không hợp lệ: {line.strip()}")

if errors:
    print("Các lỗi phát hiện được:")
    for error in errors:
        print(f"  - {error}")
else:
    print("Không phát hiện lỗi trong file nhãn.")

=== Thống kê số lượng ảnh ===
Số lượng ảnh trong train_images: 1200
Số lượng ảnh trong test_image: 300
Số lượng ảnh trong unseen_test_images: 500
Tổng số lượng ảnh: 2000

=== Thống kê nhãn ===
Số lượng file nhãn: 2000

=== Thống kê ảnh có nhãn và không có nhãn ===
Số lượng ảnh có nhãn: 2000
Số lượng ảnh không có nhãn: 0
Số lượng nhãn không có ảnh tương ứng: 0

=== Phân tích chi tiết từng thư mục ===
Thư mục train_images:
  - Số lượng ảnh có nhãn: 1200
  - Số lượng ảnh không có nhãn: 0
  - Không có ảnh nào thiếu nhãn.

Thư mục test_image:
  - Số lượng ảnh có nhãn: 300
  - Số lượng ảnh không có nhãn: 0
  - Không có ảnh nào thiếu nhãn.

Thư mục unseen_test_images:
  - Số lượng ảnh có nhãn: 500
  - Số lượng ảnh không có nhãn: 0
  - Không có ảnh nào thiếu nhãn.

=== Nhãn không có ảnh tương ứng ===
Không có nhãn nào thiếu ảnh.

=== Kiểm tra lỗi trong file nhãn ===
Không phát hiện lỗi trong file nhãn.
